In [ ]:
!pip install statsbombpy


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 1.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 8.4 MB/s eta 0:00:00


In [2]:
import warnings
from statsbombpy import sb
import pandas as pd
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [3]:
print(sb.competitions())

    competition_id  season_id   country_name        competition_name  \
0                9        281        Germany           1. Bundesliga   
1                9         27        Germany           1. Bundesliga   
2             1267        107         Africa  African Cup of Nations   
3               16          4         Europe        Champions League   
4               16          1         Europe        Champions League   
..             ...        ...            ...                     ...   
69              55         43         Europe               UEFA Euro   
70              35         75         Europe      UEFA Europa League   
71              53        106         Europe       UEFA Women's Euro   
72              72        107  International       Women's World Cup   
73              72         30  International       Women's World Cup   

   competition_gender  competition_youth  competition_international  \
0                male              False                      Fa

In [4]:
df_competiciones=sb.competitions()
competiciones_espanolas=df_competiciones[df_competiciones["country_name"]=="Spain"]
liga_20_21=competiciones_espanolas[competiciones_espanolas['season_name']=='2020/2021']
print(liga_20_21)

    competition_id  season_id country_name competition_name  \
38              11         90        Spain          La Liga   

   competition_gender  competition_youth  competition_international  \
38               male              False                      False   

   season_name               match_updated           match_updated_360  \
38   2020/2021  2024-07-15T15:15:28.367546  2024-07-15T15:17:45.455234   

           match_available_360             match_available  
38  2024-07-15T15:17:45.455234  2024-07-15T15:15:28.367546  


In [ ]:
df_matches_2021=sb.matches(competition_id=11,season_id=90)
df_matches_2021

In [43]:
def add_player_stats(players_list,stat,name_stat,player,mean):
    found=False
    for play in players_list:
        if play["name"]==player:
            list_stats=play.get("stats")
            for stat_x in list_stats:
                if stat_x['name_stat']==name_stat:
                    if mean:
                        matches_played=stat_x['Matches Played']
                        new_mean=(stat_x['stat']* matches_played + stat)/(matches_played+1)
                        stat_x['stat']=new_mean
                    found=True
                
            if not found:
                play["stats"].append({"name_stat":name_stat,"stat":stat})
        

In [44]:
def investigate_stat(players_list,name_stat,df,percen):
     
    unique_players = df['player'].unique()
    # Recorrer los jugadores únicos y contar sus apariciones
    for player in unique_players:
        #We have to calculate the % of the stat in the game
        if percen is not None:
            full_stat = (percen['player'] == player).sum()
            well_stat = (df['player'] == player).sum()
            stat=(well_stat/full_stat)*100
            add_player_stats(players_list,stat,name_stat,player,True)
        else:
            stat = (df['player'] == player).sum()
            add_player_stats(players_list,stat,name_stat,player,False)

In [45]:
def add_matches_played(players_list,event_df):
    found=False
    for player in players_list:
        name=player['name']
        if name in event_df['player'].unique():
            list_stats=player.get("stats")
            for stat_x in list_stats:
                if stat_x['name_stat']=='Matches Played':
                    stat_x['stat']+=1
                    found=True
            if not found:
                list_stats.append({'name_stat':'Matches Played','stat':1})
                     
                    


In [46]:
def remove_na(df_list):
    new_list=[]
    for df in df_list:
        df=df.dropna(axis=1,how='all')
        new_list.append(df)
    return new_list

In [47]:
def create_playerslist(team):
    player_list=[]
    for index, row in team.iterrows():
        player_name = row['player_name']
        positions = row['positions']
        for position in positions:
            if position['position']=='Left Back' or position['position']=='Right Back':
                player_list.append({'name':player_name,'stats':[]})
    return player_list

In [48]:
def printearcolumnas(df):
    print(df.columns)

In [ ]:
#Vamos a hacer una lista de los mejores laterales de la liga y sus estadísticas a lo largo de la temporada. Por cada jugador vamos a guardar: {'player','stats':[stats we want]}

fullbacks_list=[]
for match in df_matches_2021["match_id"]:
    eventos=sb.events(match_id=match)
    not_barcelona_events=eventos[eventos['team']!='Barcelona']
    lineup=sb.lineups(match_id=match)
    for team in lineup:
        if team!='Barcelona':
            fullbacks_list=create_playerslist(lineup[team])

    add_matches_played(fullbacks_list,not_barcelona_events)
    #Ball recoveries
    ball_recovery=not_barcelona_events[not_barcelona_events['type']=='Ball Recovery']
    investigate_stat(fullbacks_list,"Ball recoveries",ball_recovery[['player']],None)
    #Duels
    duels=not_barcelona_events[not_barcelona_events['type']=='Duel']
    won_duels=not_barcelona_events[not_barcelona_events['duel_outcome']=="Won"]
    investigate_stat(fullbacks_list,"Duels",won_duels[['player']],duels[['player']])
    #Pass completion %
    passes=not_barcelona_events[not_barcelona_events['type']=='Pass']
    
    every_pass=passes[['player','pass_recipient']]
    passes_completed=every_pass.dropna(subset=['pass_recipient'])
    investigate_stat(fullbacks_list,"Passes completed",passes_completed[['player']],every_pass[['player']])

    num_dribbles=not_barcelona_events[not_barcelona_events['type']=='Dribble']
    dribbles_completed=num_dribbles[num_dribbles['dribble_outcome']=='Complete']

    investigate_stat(fullbacks_list,"Num Dribbles",num_dribbles[['player']],None)
    investigate_stat(fullbacks_list,"Num Dribbles",dribbles_completed[['player']],num_dribbles[['player']])
    
    interception=eventos[eventos['type']=='Interception']
    won_interceptions=interception[(interception['interception_outcome']=='Won') | (interception['interception_outcome']=='Success In Play')]
    investigate_stat(fullbacks_list,"Interceptions",won_interceptions[['player']],None)

    #print(duels[duels['duel_outcome']=="Won"])
    #add_player_stats(players_list,)
    #print(df_list_no_nan[0])
    break
print(fullbacks_list)
#See how are the events and which ones we have to take to select the best defenders
#The types we are interested in are 'Block','Ball Recovery','Duel','Interception','Dribbled Past'  
#eventos_1=lista_eventos[0]

#eventos_1['type'].unique()
#duelos=eventos_1[eventos_1['type']=='Duel']
#duelos.columns
#duelos=duelos[['player','team','duel_type','duel_outcome']]
#print(duelos['duel_outcome'].unique())
#jugador_con_masduelos_ganados=duelos[duelos['duel_outcome']=='Won'].value_counts(['player'])
#jugador_con_masduelos_ganados


[{'name': 'Rubén Duarte Sánchez', 'stats': [{'name_stat': 'Matches Played', 'stat': 1}, {'name_stat': 'Ball recoveries', 'stat': 1}, {'name_stat': 'Duels', 'stat': 33.33333333333333}, {'name_stat': 'Passes completed', 'stat': 80.0}, {'name_stat': 'Interceptions', 'stat': 2}]}, {'name': 'Martín Aguirregabiria Padilla', 'stats': [{'name_stat': 'Matches Played', 'stat': 1}, {'name_stat': 'Passes completed', 'stat': 100.0}, {'name_stat': 'Interceptions', 'stat': 1}]}, {'name': 'Joaquín Navarro Jiménez', 'stats': [{'name_stat': 'Matches Played', 'stat': 1}, {'name_stat': 'Ball recoveries', 'stat': 1}, {'name_stat': 'Duels', 'stat': 16.666666666666664}, {'name_stat': 'Passes completed', 'stat': 86.36363636363636}]}]


In [8]:
for comp in sb.competitions()["country_name"]:
    print(comp)

1. Bundesliga
1. Bundesliga
African Cup of Nations
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Champions League
Copa America
Copa del Rey
Copa del Rey
Copa del Rey
FA Women's Super League
FA Women's Super League
FA Women's Super League
FIFA U20 World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
FIFA World Cup
Indian Super league
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
La Liga
Liga Profesional
Liga Profesional
Ligue 1
Ligue 1
Ligue 1
Major League Soccer
North American League
NWSL
Premier League
Premier League
Serie A
Serie A
UEFA Euro
UEFA Euro
UEFA Europa League
UEFA Women's Euro
Women's 

/home/ale/.local/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
